In [1]:
import os
import csv
from pprint import pprint
import pandas as pd
import pickle
import collections

In [2]:
hashPath = "/Users/Bya/git/EPLdata/"
dataPath = "/Users/Bya/Dropbox/Research/datas/"
modelPath = "/Users/Bya/Dropbox/Research/datas/Pickles/Models/"
tokenizerPath = "/Users/Bya/git/predictEPL/Tokenizers/"

os.chdir(tokenizerPath)
import sentiment_aware as sa
os.chdir(hashPath)
from config import Hashtags

In [3]:
model = "MovieShortReview"

# read saved classifier
saved_classifier_f = open(modelPath + model + "/naiveBayes.pickle", "rb")
saved_classifier = pickle.load(saved_classifier_f)
saved_classifier_f.close()

# read saved word features
word_features5k_f = open(modelPath + model + "/word_features5k.pickle", "rb")
word_features = pickle.load(word_features5k_f)
word_features5k_f.close()

# Define Sentiment Tokenizer Class as tok
tok = sa.Tokenizer(preserve_case=False)
def find_features(document):
    words = tok.tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

# Define Sentiment Analysis function. Using aboves
def sentiment(text):
    return saved_classifier.classify(find_features(text))

In [4]:
def whichSide(tags, tags_home, tags_away):
    tags = map(lambda tag: '#' + tag.lower(), tags)
    if set(tags).intersection(tags_home) and set(tags).intersection(tags_away):
        return 'both'
    elif set(tags).intersection(tags_home):
        return 'home'
    elif set(tags).intersection(tags_away):
        return 'away'
    else:
        return 'nothing'

In [5]:
def ReadFileAsDF(fileName, home_team, away_team):
    # both team hashtags
    tags_home = Hashtags.dic[home_team]
    tags_away = Hashtags.dic[away_team]

    # read file as dataframe, and add 'side' column that shows which team's tweet it is 
    dfTweets = pd.read_csv(fileName, header=None, names=['date', 'text', 'user', 'tags'])
    dfTweets['side'] = map(lambda tags: whichSide(tags, tags_home, tags_away), map(lambda tag: tag.split(','), dfTweets['tags']))
    
    # count tweets
    sides = ['home', 'away', 'both', 'nothing']
    numSides =  map(lambda side: (side, len(dfTweets[dfTweets["side"] == side])), sides)
    print numSides
    
    dfHomeTweets = dfTweets[dfTweets["side"] == 'home']
    dfAwayTweets = dfTweets[dfTweets["side"] == 'away']
    dfHomeTweets = pd.DataFrame(dfHomeTweets.values, range(len(dfHomeTweets)), dfHomeTweets.columns)
    dfAwayTweets = pd.DataFrame(dfAwayTweets.values, range(len(dfAwayTweets)), dfAwayTweets.columns)
    
    return dfHomeTweets, dfAwayTweets

In [6]:
# Create DataFrames for home and away tweets
def SentimentHomeAway(dfHomeTweets, dfAwayTweets):
    home_sent = map(lambda tweet: sentiment(tweet), dfHomeTweets["text"])
    away_sent = map(lambda tweet: sentiment(tweet), dfAwayTweets["text"])
    
    return [collections.Counter(home_sent)['pos'], 
            collections.Counter(home_sent)['neg'],
            collections.Counter(away_sent)['pos'],
            collections.Counter(away_sent)['neg']]

In [7]:
# Save the dictionary values to file
def ListSaveToCSV(fileName, myList):
    with open(fileName + '.csv', 'a') as f:
        writer = csv.writer(f, quoting=csv.QUOTE_ALL)
        writer.writerow(myList)

In [8]:
def GameNegPos(game, loadFileName, saveFileName):
    
    home_team = game[0]
    away_team = game[1]

    dfHome, dfAway = ReadFileAsDF(loadFileName, home_team, away_team)
    
    listResSent = SentimentHomeAway(dfHome, dfAway)
    listResGame = [home_team, away_team] + listResSent

    os.chdir("/Users/Bya/Dropbox/Research/datas/Results/")
    ListSaveToCSV(saveFileName, listResGame)

In [9]:
listGamesW4 = [('Newcastle', 'Arsenal'),
               
               ('Villa', 'Sunderland'),
               ('Bournemouth', 'Leicester'),
               ('Chelsea', 'Crystal'),
               ('Liverpool', 'WestHam'),
               ('City', 'Watford'),
               ('Stoke', 'WestBromwich'),
               
#                ('Tottenham', 'Everton'),   ====> not data form this game
               ('Southampton', 'Norwich'),
               ('Swansea', 'United')]

loadFileNameGW4 = ["/Users/Bya/Dropbox/Research/datas/GW4/1508292045_NUFCvsARSENAL_json.txt.csv",
                   
                   "/Users/Bya/Dropbox/Research/datas/GW4/1508292300_6matches_json.txt.csv",
                   "/Users/Bya/Dropbox/Research/datas/GW4/1508292300_6matches_json.txt.csv",
                   "/Users/Bya/Dropbox/Research/datas/GW4/1508292300_6matches_json.txt.csv",
                   "/Users/Bya/Dropbox/Research/datas/GW4/1508292300_6matches_json.txt.csv",
                   "/Users/Bya/Dropbox/Research/datas/GW4/1508292300_6matches_json.txt.csv",
                   "/Users/Bya/Dropbox/Research/datas/GW4/1508292300_6matches_json.txt.csv",
                   
                   "/Users/Bya/Dropbox/Research/datas/GW4/1508302130_SAINTSvsNCFC_json.txt.csv",
                   "/Users/Bya/Dropbox/Research/datas/GW4/1508310000_SWANSvsMUFC_json.txt.csv",]

saveFileNameGW4 = 'GW4'

for i in range(len(listGamesW4)):
    GameNegPos(listGamesW4[i], loadFileNameGW4[i], saveFileNameGW4)

[('home', 6257), ('away', 6661), ('both', 3573), ('nothing', 33)]
[('home', 628), ('away', 4), ('both', 1463), ('nothing', 26874)]
[('home', 830), ('away', 349), ('both', 766), ('nothing', 27024)]
[('home', 4350), ('away', 478), ('both', 1499), ('nothing', 22642)]
[('home', 8621), ('away', 1611), ('both', 3298), ('nothing', 15439)]
[('home', 2354), ('away', 361), ('both', 810), ('nothing', 25444)]
[('home', 1128), ('away', 406), ('both', 1294), ('nothing', 26141)]
[('home', 1806), ('away', 1634), ('both', 1835), ('nothing', 20)]
[('home', 1286), ('away', 24353), ('both', 3610), ('nothing', 64)]


In [84]:
# 'Bournemouth'
# 'Arsenal'
# 'Villa'
# 'Chelsea'
# 'Crystal'
# 'Everton'
# 'Leicester'
# 'Liverpool'
# 'City'
# 'United'
# 'Newcastle'
# 'Norwich'
# 'Southampton'
# 'Stoke'
# 'Sunderland'
# 'Swansea'
# 'Tottenham'
# 'Watford'
# 'WestBromwich'
# 'WestHam'